## 概述

与带有Attention机制的Seq2Seq所不同的是，Transformer模型是一个纯基于自注意力机制（self-attention mechanism）的架构，不包含任何循环神经网络（Recurrent Neural Network， RNN）和卷积神经网络（Convolutional Neural Network，CNN）。整个模型由
## 数据准备

我们本次使用的数据集为**Multi30K数据集**，它是一个大规模的图像-文本数据集，包含30K+图片，每张图片对应两类不同的文本描述：
- 英语描述，及对应的德语翻译；
- 五个独立的、非翻译而来的英语和德语描述，描述中包含的细节并不相同；

因其收集的不同语言对于图片的描述相互独立，所以训练出的模型可以更好地适用于有噪声的多模态内容。

![avatar](./assets/Multi30K.png)
> 图片来源：Elliott, D., Frank, S., Sima’an, K., & Specia, L. (2016). Multi30K: Multilingual English-German Image Descriptions. CoRR, 1605.00459.

首先，我们需要下载如下依赖：

- 分词工具：`pip install spacy`
- 德语/英语分词器：`python -m spacy download de_core_news_sm`，`python -m spacy download en_core_web_sm`

### 数据下载模块

使用`download`进行数据下载，并将`tar.gz`文件解压到指定文件夹。

下载好的数据集目录结构如下：

```text
home_path/.mindspore_examples
├─test
│      test2016.de
│      test2016.en
│      test2016.fr
│
├─train
│      train.de
│      train.en
│
└─valid
        val.de
        val.en
```

In [1]:
from download import download
from pathlib import Path
from tqdm import tqdm
import os

# 训练、验证、测试数据集下载地址
urls = {
    'train': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz',
    'valid': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz',
    'test': 'http://www.quest.dcs.shef.ac.uk/wmt17_files_mmt/mmt_task1_test2016.tar.gz'
}

# 指定保存路径为 `home_path/.mindspore_examples`
cache_dir = Path.home() / '.mindspore_examples'

train_path = download(urls['train'], os.path.join(cache_dir, 'train'), kind='tar.gz')
valid_path = download(urls['valid'], os.path.join(cache_dir, 'valid'), kind='tar.gz')
test_path = download(urls['test'], os.path.join(cache_dir, 'test'), kind='tar.gz')

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.


### 数据预处理

在使用数据进行模型训练等操作时，我们需要对数据进行预处理，流程如下：

1. 加载数据集，目前数据为句子形式的文本，需要进行分词，即将句子拆解为单独的词元（token，可以为字符或者单词）；
    - 分词可以使用`spaCy`创建分词器（tokenizer）：`de_core_news_sm`，`en_core_web_sm`，需要手动下载；
    - 分词后，去除多余的空格，统一大小写等；
2. 将每个词元映射到从0开始的数字索引中（为节约存储空间，可过滤掉词频低的词元），词元和数字索引所构成的集合叫做词典（vocabulary）；
3. 添加特殊占位符，标明序列的起始与结束，统一序列长度，并创建数据迭代器；

#### 数据加载器

In [2]:
import spacy
from functools import partial

class Multi30K():
    """Multi30K数据集加载器

    加载Multi30K数据集并处理为一个Python迭代对象。

    """
    def __init__(self, path):
        self.data = self._load(path)

    def _load(self, path):
        def tokenize(text, spacy_lang):
            # 去除多余空格，统一大小写
            text = text.rstrip()
            return [tok.text.lower() for tok in spacy_lang.tokenizer(text)]

        # 加载英、德语分词器
        tokenize_de = partial(tokenize, spacy_lang=spacy.load('de_core_news_sm'))
        tokenize_en = partial(tokenize, spacy_lang=spacy.load('en_core_web_sm'))

        # 读取Multi30K数据，并进行分词
        members = {i.split('.')[-1]: i for i in os.listdir(path)}
        de_path = os.path.join(path, members['de'])
        en_path = os.path.join(path, members['en'])
        with open(de_path, 'r') as de_file:
            de = de_file.readlines()[:-1]
            de = [tokenize_de(i) for i in de]
        with open(en_path, 'r') as en_file:
            en = en_file.readlines()[:-1]
            en = [tokenize_en(i) for i in en]

        return list(zip(de, en))

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

In [3]:
train_dataset, valid_dataset, test_dataset = Multi30K(train_path), Multi30K(valid_path), Multi30K(test_path)

对解压和分词结果进行测试，打印测试数据集第一组英德语文本，可以看到每一个单词和标点符号已经被单独分离出来。

In [4]:
for de, en in test_dataset:
    print(f'de = {de}')
    print(f'en = {en}')
    break

de = ['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.']
en = ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


#### 词典

In [5]:
class Vocab:
    """通过词频字典，构建词典"""

    special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']

    def __init__(self, word_count_dict, min_freq=1):
        self.word2idx = {}
        for idx, tok in enumerate(self.special_tokens):
            self.word2idx[tok] = idx

        # 过滤低词频的词元
        filted_dict = {
            w: c
            for w, c in word_count_dict.items() if c >= min_freq
        }
        for w, _ in filted_dict.items():
            self.word2idx[w] = len(self.word2idx)

        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.bos_idx = self.word2idx['<bos>']  # 特殊占位符：序列开始
        self.eos_idx = self.word2idx['<eos>']  # 特殊占位符：序列结束
        self.pad_idx = self.word2idx['<pad>']  # 特殊占位符：补充字符
        self.unk_idx = self.word2idx['<unk>']  # 特殊占位符：低词频词元或未曾出现的词元

    def _word2idx(self, word):
        """单词映射至数字索引"""
        if word not in self.word2idx:
            return self.unk_idx
        return self.word2idx[word]

    def _idx2word(self, idx):
        """数字索引映射至单词"""
        if idx not in self.idx2word:
            raise ValueError('input index is not in vocabulary.')
        return self.idx2word[idx]

    def encode(self, word_or_list):
        """将单个单词或单词数组映射至单个数字索引或数字索引数组"""
        if isinstance(word_or_list, list):
            return [self._word2idx(i) for i in word_or_list]
        return self._word2idx(word_or_list)

    def decode(self, idx_or_list):
        """将单个数字索引或数字索引数组映射至单个单词或单词数组"""
        if isinstance(idx_or_list, list):
            return [self._idx2word(i) for i in idx_or_list]
        return self._idx2word(idx_or_list)

    def __len__(self):
        return len(self.word2idx)

通过自定义词频字典进行测试，我们可以看到词典已去除词频少于2的词元c，并加入了默认的四个特殊占位符，故词典整体长度为：4 - 1 + 4 = 7

In [6]:
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)

7

使用`collections`中的`Counter`和`OrderedDict`统计英/德语每个单词在整体文本中出现的频率。构建词频字典，然后再将词频字典转为词典。

在分配数字索引时有一个小技巧：常用的词元对应数值较小的索引，这样可以节约空间。

In [7]:
from collections import Counter, OrderedDict

def build_vocab(dataset):
    de_words, en_words = [], []
    for de, en in dataset:
        de_words.extend(de)
        en_words.extend(en)

    de_count_dict = OrderedDict(sorted(Counter(de_words).items(), key=lambda t: t[1], reverse=True))
    en_count_dict = OrderedDict(sorted(Counter(en_words).items(), key=lambda t: t[1], reverse=True))

    return Vocab(de_count_dict, min_freq=2), Vocab(en_count_dict, min_freq=2)

In [8]:
de_vocab, en_vocab = build_vocab(train_dataset)
print('Unique tokens in de vocabulary:', len(de_vocab))

Unique tokens in de vocabulary: 7853


#### 数据迭代器

数据预处理的最后一步是创建数据迭代器，我们在进一步处理数据（包括批处理，添加起始和终止符号，统一序列长度）后，将数据以张量的形式返回。

创建数据迭代器需要如下参数：

- `dataset`：分词后的数据集
- `de_vocab`：德语词典
- `en_vocab`：英语词典
- `batch_size`：批量大小，即一个batch中包含多少个序列
- `max_len`：序列最大长度，为最长有效文本长度 + 2（序列开始、序列结束占位符），如不满则补齐，如超过则丢弃
- `drop_remainder`：是否在最后一个batch未满时，丢弃该batch

In [9]:
import mindspore

class Iterator():
    """创建数据迭代器"""
    def __init__(self, dataset, de_vocab, en_vocab, batch_size, max_len=32, drop_reminder=False):
        self.dataset = dataset
        self.de_vocab = de_vocab
        self.en_vocab = en_vocab

        self.batch_size = batch_size
        self.max_len = max_len
        self.drop_reminder = drop_reminder

        length = len(self.dataset) // batch_size
        self.len = length if drop_reminder else length + 1  # 批量数量

    def __call__(self):
        def pad(idx_list, vocab, max_len):
            """统一序列长度，并记录有效长度"""
            idx_pad_list, idx_len = [], []
            # 当前序列度超过最大长度时，将超出的部分丢弃；当前序列长度小于最大长度时，用占位符补齐
            for i in idx_list:
                if len(i) > max_len - 2:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i[:max_len-2] + [vocab.eos_idx]
                    )
                    idx_len.append(max_len)
                else:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i + [vocab.eos_idx] + [vocab.pad_idx] * (max_len - len(i) - 2)
                    )
                    idx_len.append(len(i) + 2)
            return idx_pad_list, idx_len

        def sort_by_length(src, trg):
            """对德/英语的字段长度进行排序"""
            data = zip(src, trg)
            data = sorted(data, key=lambda t: len(t[0]), reverse=True)
            return zip(*list(data))

        def encode_and_pad(batch_data, max_len):
            """将批量中的文本数据转换为数字索引，并统一每个序列的长度"""
            # 将当前批量数据中的词元转化为索引
            src_data, trg_data = zip(*batch_data)
            src_idx = [self.de_vocab.encode(i) for i in src_data]
            trg_idx = [self.en_vocab.encode(i) for i in trg_data]

            # 统一序列长度
            src_idx, trg_idx = sort_by_length(src_idx, trg_idx)
            src_idx_pad, src_len = pad(src_idx, de_vocab, max_len)
            trg_idx_pad, _ = pad(trg_idx, en_vocab, max_len)

            return src_idx_pad, src_len, trg_idx_pad

        for i in range(self.len):
            # 获取当前批量的数据
            if i == self.len - 1 and not self.drop_reminder:
                batch_data = self.dataset[i * self.batch_size:]
            else:
                batch_data = self.dataset[i * self.batch_size: (i+1) * self.batch_size]

            src_idx, src_len, trg_idx = encode_and_pad(batch_data, self.max_len)
            # 将序列数据转换为tensor
            yield mindspore.Tensor(src_idx, mindspore.int32), \
                mindspore.Tensor(src_len, mindspore.int32), \
                mindspore.Tensor(trg_idx, mindspore.int32)

    def __len__(self):
        return self.len

In [10]:
train_iterator = Iterator(train_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=True)
valid_iterator = Iterator(valid_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)
test_iterator = Iterator(test_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)

## 模型解析

### Transformer 架构

与Seq2Seq类似，编码器+解码器+中间信息传递

### 位置编码（Positional Encoding）

### 多头注意力（Multi-Head Attention）

- 自注意力
- 多头注意力
- 带掩码的多头注意力

### Add & Norm

- Add
- Norm： Layer Norm

### 前馈神经网络（Feed-Forward Nerual Network，FFN）

- Pointwise FFN

### 编码器（Encoder）

### 解码器（Decoder）

In [11]:
import mindspore.nn as nn

d_model = 512  # Embedding层维度
n_head = 8  # 多头感知机中头的数量
n_layer = 6  # 编码器和解码器的层数
d_ff = 2048  # 前馈神经网络维度
max_len = 32  # 序列最大长度


model = nn.Transformer()

loss_fn = nn.CrossEntropyLoss()
opt = nn.Adam(model.trainable_params(), learning_rate=0.0001)

def forward_fn(src, trg):
    output = model(src, trg)
    loss = loss_fn(output, trg)

    return loss

grad_fn = mindspore.value_and_grad(forward_fn, None, opt.parameters)

def train_step(src, trg):
    loss, grads = grad_fn(src, trg)
    opt(grads)
    return loss

In [12]:
from tqdm import tqdm
from mindspore import Tensor
import numpy as np

def get_sinusoid_encoding_table(n_position, d_model):
    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_model)
    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
    return Tensor(sinusoid_table, mindspore.float32)

def embedding(input, vocab, max_len, d_model):
    context_emb = nn.Embedding(len(vocab), d_model)
    sinusoid_table = get_sinusoid_encoding_table(max_len, d_model)
    pos_emb = nn.Embedding(sinusoid_table.shape[0], sinusoid_table.shape[1], embedding_table=sinusoid_table)
    output = context_emb(input) + pos_emb(input)
    return output


In [ ]:
def train(iterator, epoch=0):
    """模型训练"""
    model.set_train(True)
    num_batches = len(iterator)
    total_loss = 0 # 所有batch训练loss的累加
    total_steps = 0 # 训练步数

    with tqdm(total=num_batches) as t:
        t.set_description(f'Epoch: {epoch}')
        for src, src_len, trg in iterator():
            src_emb = embedding(src, de_vocab, max_len, d_model)
            trg_emb = embedding(trg, en_vocab, max_len, d_model)
            # print(src_emb.shape)
            # print(trg_emb.shape)
            loss = train_step(src_emb, trg_emb) # 当前batch的loss
            total_loss += loss.asnumpy()
            total_steps += 1
            curr_loss = total_loss / total_steps # 当前的平均loss
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)
    
    return total_loss / total_steps


def evaluate(iterator):
    """模型验证"""
    model.set_train(False)
    num_batches = len(iterator)
    total_loss = 0 # 所有batch训练loss的累加
    total_steps = 0 # 训练步数
    
    with tqdm(total=num_batches) as t:
        for src, src_len, trg in iterator():
            src_emb = embedding(src, de_vocab, max_len, d_model)
            trg_emb = embedding(trg, en_vocab, max_len, d_model)
            loss = forward_fn(src_emb, trg_emb) # 当前batch的loss
            total_loss += loss.asnumpy()
            total_steps += 1
            curr_loss = total_loss / total_steps # 当前的平均loss
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)
    
    return total_loss / total_steps

In [13]:
from mindspore import save_checkpoint

num_epochs = 10 # 训练迭代数
clip = 1.0 # 梯度裁剪阈值
best_valid_loss = float('inf') # 当前最佳验证损失
ckpt_file_name = os.path.join(cache_dir, 'seq2seq.ckpt') # 模型保存路径

mindspore.set_context(mode=mindspore.PYNATIVE_MODE)
# mindspore.set_context(mode=mindspore.GRAPH_MODE)

for i in range(num_epochs):
    # 模型训练，网络权重更新
    train_loss = train(train_iterator, i)
    # 网络权重更新后对模型进行验证
    valid_loss = evaluate(valid_iterator)
    
    # 保存当前效果最好的模型
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        save_checkpoint(model, ckpt_file_name)

Epoch: 0:   0%|          | 0/226 [00:00<?, ?it/s]


TypeError: For Primitive[Dtype], the input argument[input_x] must be a Tensor, CSRTensor or COOTensor, but got <class 'tuple'>.

In [17]:
net_work = nn.Transformer()

src = Tensor(np.random.rand(128, 32, 512), mindspore.float32)
trg = Tensor(np.random.rand(128, 32, 512), mindspore.float32)

def forward_fn(src, trg):
    out = net_work(src, trg)
    loss = loss_fn(out, trg)
    return loss

optimizer = nn.Adam(net_work.trainable_params(), learning_rate=0.0001)
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters)

def train_step(src, trg):
    loss, grads = grad_fn(src, trg)
    optimizer(grads)

net_work.set_train(True)   
train_step(src, trg)


TypeError: For Primitive[Dtype], the input argument[input_x] must be a Tensor, CSRTensor or COOTensor, but got <class 'tuple'>.

In [20]:
net_work = nn.Transformer()

src = Tensor(np.random.rand(128, 32, 512), mindspore.float32)
trg = Tensor(np.random.rand(128, 32, 512), mindspore.float32)

def forward_fn(src, trg):
    out = net_work(src, trg)
    loss = loss_fn(out, trg)
    return loss

optimizer = nn.Adam(net_work.trainable_params(), learning_rate=0.0001)
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters)

def train_step(src, trg):
    loss, grads = grad_fn(src, trg)
    optimizer(grads)

# net_work.set_train(True)   
train_step(src, trg)